In [1]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
path = "qst_data/z0_linear_20200909_114932.pkl"

with open(path, 'rb') as f:
    param_linear_est_linear = pickle.load(f)

In [4]:
len(param_linear_est_linear)

1000

In [8]:
for est in param_linear_est_linear:
    print(est[0].calc_eigenvalues())

[ 1.00004500e+00+0.j -4.49979752e-05+0.j]
[ 1.00072148e+00+1.8617983e-19j -7.21479467e-04-1.8617983e-19j]
[ 1.00070850e+00-4.61252163e-20j -7.08498031e-04+4.61252163e-20j]
[ 1.0013033+1.25007177e-19j -0.0013033-1.25007177e-19j]
[ 1.00097605e+00-1.08233219e-19j -9.76047332e-04+1.08233219e-19j]
[ 1.00017797e+00+2.3135863e-20j -1.77968327e-04-2.3135863e-20j]
[ 1.00006500e+00+0.j -6.49957755e-05+0.j]
[ 1.00038585e+00+1.70278139e-20j -3.85851119e-04-1.70278139e-20j]
[ 1.00093113e+00+1.05720738e-19j -9.31132991e-04-1.05720738e-19j]
[ 1.00111276+2.31082674e-19j -0.00111276-2.31082674e-19j]
[ 1.00004100e+00+1.11067392e-20j -4.09983191e-05-1.11067392e-20j]
[ 1.00074445e+00-9.45566879e-20j -7.44445800e-04+9.45566879e-20j]
[ 1.00038885e+00-1.02562522e-19j -3.88848797e-04+1.02562522e-19j]
[ 1.00010099e+00+0.j -1.00989801e-04+0.j]
[ 1.00006800e+00+0.j -6.79953766e-05+0.j]
[ 1.00010599e+00+8.92814309e-21j -1.05988766e-04-8.92814309e-21j]
[ 1.00005200e+00-1.28169903e-36j -5.19972963e-05-2.46506216e-3

[ 1.00005000e+00-1.53314011e-21j -4.99975003e-05+1.53314011e-21j]
[ 1.00019396e+00+1.20762664e-20j -1.93962379e-04-1.20762664e-20j]
[ 1.00005200e+00+1.28169903e-36j -5.19972963e-05+2.46506216e-32j]
[ 1.00054071e+00-4.03050523e-20j -5.40707635e-04+4.03050523e-20j]
[ 1.00005200e+00+6.25398406e-21j -5.19972963e-05-6.25398406e-21j]
[ 1.00040883e+00-2.51756848e-36j -4.08832856e-04-6.16045825e-33j]
[ 1.00115167+0.j -0.00115167+0.j]
[ 1.00022495e+00-2.60091507e-20j -2.24949398e-04+2.60091507e-20j]
[ 1.00024494e+00+2.71394222e-20j -2.44940004e-04-2.71394222e-20j]
[ 1.00069651e+00+9.14686094e-20j -6.96514867e-04-9.14686094e-20j]
[ 1.00004500e+00-2.90895792e-21j -4.49979752e-05+2.90895792e-21j]
[ 1.0000130e+00-1.56361796e-21j -1.2999831e-05+1.56361796e-21j]
[ 1.00016197e+00-2.20722749e-20j -1.61973764e-04+2.20722749e-20j]
[ 1.0000130e+00-1.56361796e-21j -1.2999831e-05+1.56361796e-21j]
[ 1.00078738e+00-9.68994886e-36j -7.87380033e-04-1.23162617e-32j]
[ 1.00006500e+00+0.j -6.49957755e-05+0.j]
[ 1.

[ 1.00038785e+00+0.j -3.87849573e-04+0.j]
[ 1.00261914+6.42303321e-35j -0.00261914+2.45876730e-32j]
[ 1.00008899e+00-8.18121814e-21j -8.89920804e-05+8.18121814e-21j]
[ 1.00008499e+00+0.j -8.49927762e-05+0.j]
[ 1.00042482e+00-2.61592989e-36j -4.24819528e-04-6.16035989e-33j]
[ 1.00094810e+00+1.06676959e-19j -9.48101104e-04-1.06676959e-19j]
[ 1.0000260e+00-1.10561612e-21j -2.5999324e-05+1.10561612e-21j]
[ 1.0000170e+00+5.23826230e-38j -1.6999711e-05+3.08143553e-33j]
[ 1.00032489e+00+3.90660806e-21j -3.24894444e-04-3.90660806e-21j]
[ 1.00014498e+00-2.08827817e-20j -1.44978981e-04+2.08827817e-20j]
[ 1.00071948e+00-8.85556175e-36j -7.19482345e-04-1.23170961e-32j]
[ 1.00055669e+00-8.17908239e-20j -5.56690096e-04+8.17908239e-20j]
[ 1.00049975e+00+1.93754323e-20j -4.99750250e-04-1.93754323e-20j]
[ 1.00025993e+00+0.j -2.59932435e-04+0.j]
[ 1.00025593e+00+0.j -2.55934498e-04+0.j]
[ 1.00037686e+00+0.j -3.76857978e-04+0.j]
[ 1.00132325-2.51912864e-19j -0.00132325+2.51912864e-19j]
[ 1.00192928+0.j -

[ 1.00008499e+00+0.j -8.49927762e-05+0.j]
[ 1.00022095e+00+5.44446121e-36j -2.20951181e-04+2.46464588e-32j]
[ 1.00020496e+00+2.48272832e-20j -2.04957992e-04-2.48272832e-20j]
[ 1.00098503e+00+1.08728642e-19j -9.85029716e-04-1.08728642e-19j]
[ 1.00015698e+00+2.1729201e-20j -1.56975359e-04-2.1729201e-20j]
[ 1.0000260e+00+1.10561612e-21j -2.5999324e-05-1.10561612e-21j]
[ 1.00060963e+00+1.50101979e-35j -6.09628353e-04+2.46368931e-32j]
[ 1.00003700e+00+0.j -3.69986311e-05+0.j]
[ 1.00024994e+00+0.j -2.49937531e-04+0.j]
[ 1.00035587e+00-3.27074177e-20j -3.55873354e-04+3.27074177e-20j]
[ 1.00036487e+00+4.49405196e-36j -3.64866872e-04+1.23214576e-32j]
[ 1.00013698e+00+3.37592331e-36j -1.36981236e-04+2.46485274e-32j]
[ 1.00012099e+00+0.j -1.20985363e-04+0.j]
[ 1.00056168e+00-8.21562883e-20j -5.61684511e-04+8.21562883e-20j]
[ 1.00296918-1.81909841e-35j -0.00296918-6.14478484e-33j]
[ 1.00083131e+00+2.04593312e-35j -8.31308925e-04+2.46314440e-32j]
[ 1.00032390e+00+0.j -3.23895092e-04+0.j]
[ 1.000753

[ 1.00006500e+00+0.j -6.49957755e-05+0.j]
[ 1.00031390e+00+3.86670676e-36j -3.13901466e-04+1.23220849e-32j]
[ 1.00046578e+00+1.87063404e-20j -4.65783046e-04-1.87063404e-20j]
[ 1.00063659e+00-8.74535373e-20j -6.36594747e-04+8.74535373e-20j]
[ 1.00008099e+00+0.j -8.09934401e-05+0.j]
[ 1.00064059e+00+3.94288695e-36j -6.40589645e-04+6.15903294e-33j]
[ 1.00019596e+00+0.j -1.95961599e-04+0.j]
[ 1.00015997e+00-1.09678454e-20j -1.59974408e-04+1.09678454e-20j]
[ 1.00072947e+00-9.36027343e-20j -7.29467877e-04+9.36027343e-20j]
[ 1.00030491e+00+1.51385898e-20j -3.04907032e-04-1.51385898e-20j]
[ 1.00033289e+00-7.9086678e-21j -3.32889185e-04+7.9086678e-21j]
[ 1.00124744+1.83463875e-19j -0.00124744-1.83463875e-19j]
[ 1.00020196e+00-4.97666881e-36j -2.01959212e-04-2.46469266e-32j]
[ 1.00022095e+00-2.72223061e-36j -2.20951181e-04-1.23232294e-32j]
[ 1.00028992e+00+2.95241573e-20j -2.89915949e-04-2.95241573e-20j]
[ 1.00007199e+00+1.77455374e-36j -7.19948167e-05+2.46501287e-32j]
[ 1.00057767e+00+0.j -5.77

In [13]:
i = 184
print(i)
param_linear_est_linear[i][0].calc_eigenvalues()

184


array([ 1.00000000e+00+0.j, -2.22044605e-16+0.j])

In [22]:
i = 184
print(i)
param_linear_est_linear[i][0].to_density_matrix()

184


array([[ 1.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j, -2.22044605e-16+0.j]])

In [14]:
i = 420
print(i)
param_linear_est_linear[i][0].calc_eigenvalues()

420


array([ 1.00000000e+00+0.j, -2.22044605e-16+0.j])

In [23]:
i = 420
print(i)
param_linear_est_linear[i][0].to_density_matrix()

420


array([[ 1.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j, -2.22044605e-16+0.j]])

In [35]:
def to_density_matrix_for_debug(state) -> np.ndarray:
        density = np.zeros((state._dim, state._dim), dtype=np.complex128)
        density_list = []
        for coefficient, basis in zip(state._vec, state.composite_system.basis()):
            tmp = coefficient * basis
            print(tmp)
            density_list.append(tmp)
        
        return density_list

In [36]:
np.linalg.eigvals(.to_density_matrix())

SyntaxError: invalid syntax (<ipython-input-36-119a7422f584>, line 1)

In [40]:
print(f"i={i}")
target_state = param_linear_est_linear[i][0] 
print("vec=")
print(target_state.vec)
print()
print("density_matrix=sum(")
density_list = to_density_matrix_for_debug(target_state)
print(")")

i=420
vec=
[0.70710678 0.         0.         0.70710678]

density_matrix=sum(
[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[ 0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
)


In [41]:
density_list[0] + density_list[-1]

array([[ 1.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j, -2.22044605e-16+0.j]])

In [43]:
np.array([[0.5, 0], [0, 0.5]], dtype=np.complex128) + np.array([[0.5, 0], [0, -0.5]], dtype=np.complex128)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [44]:
density_list[-1][1][1]

(-0.5000000000000001+0j)

In [46]:
density_list[-1][1][1].real == -0.5

False

In [49]:
for i, density in enumerate(density_list):
    print(f"{i}----------------------")
    print(f"(0, 0) = {density[0][0]}")
    print(f"(0, 1) = {density[0][1]}")
    print(f"(1, 0) = {density[1][0]}")
    print(f"(1, 1) = {density[1][1]}")

0----------------------
(0, 0) = (0.4999999999999999+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (0.4999999999999999+0j)
1----------------------
(0, 0) = 0j
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = 0j
2----------------------
(0, 0) = 0j
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = 0j
3----------------------
(0, 0) = (0.5000000000000001+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (-0.5000000000000001+0j)


In [52]:
-0.5000000000000001+0j + 0.4999999999999999+0j

(-2.220446049250313e-16+0j)

In [53]:
0.4999999999999999+0j + 0.5000000000000001+0j

(1+0j)

In [57]:
(-5000000000000001 + 4999999999999999) / 10 ** 16

-2e-16

In [56]:
4999999999999999 + 5000000000000001

10000000000000000

In [63]:
total = np.zeros((2, 2), dtype=complex)
for i, density in enumerate(density_list):
    density = density * 10 ** 16
    print(f"{i}----------------------")
    print(f"(0, 0) = {density[0][0]}")
    print(f"(0, 1) = {density[0][1]}")
    print(f"(1, 0) = {density[1][0]}")
    print(f"(1, 1) = {density[1][1]}")
    
    total += density

0----------------------
(0, 0) = (4999999999999999+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (4999999999999999+0j)
1----------------------
(0, 0) = 0j
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = 0j
2----------------------
(0, 0) = 0j
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = 0j
3----------------------
(0, 0) = (5000000000000001+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (-5000000000000001+0j)


In [65]:
a = total / 10**16

print(f"(0, 0) = {a[0][0]}")
print(f"(0, 1) = {a[0][1]}")
print(f"(1, 0) = {a[1][0]}")
print(f"(1, 1) = {a[1][1]}")

(0, 0) = (1+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (-2e-16+0j)


In [69]:
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])
true_object = get_z0_1q(c_sys)
print("density_matrix=sum(")
true_density_list = to_density_matrix_for_debug(true_object)
print(")")

density_matrix=sum(
[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
[[ 0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
)


In [70]:
for i, density in enumerate(true_density_list):
    # density = density * 10 ** 16
    print(f"{i}----------------------")
    print(f"(0, 0) = {density[0][0]}")
    print(f"(0, 1) = {density[0][1]}")
    print(f"(1, 0) = {density[1][0]}")
    print(f"(1, 1) = {density[1][1]}")

0----------------------
(0, 0) = (0.49999999999999983+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (0.49999999999999983+0j)
1----------------------
(0, 0) = 0j
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = 0j
2----------------------
(0, 0) = 0j
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = 0j
3----------------------
(0, 0) = (0.49999999999999983+0j)
(0, 1) = 0j
(1, 0) = 0j
(1, 1) = (-0.49999999999999983+0j)


In [71]:
true_object.vec

array([0.70710678, 0.        , 0.        , 0.70710678])

In [80]:
print(420)
for i, v in enumerate(target_state.vec):
    print(f"vec[{i}] = {v}")

420
vec[0] = 0.7071067811865475
vec[1] = 0.0
vec[2] = 0.0
vec[3] = 0.7071067811865477


In [81]:
print("true_object:")
for i, v in enumerate(true_object.vec):
    print(f"vec[{i}] = {v}")

true_object:
vec[0] = 0.7071067811865474
vec[1] = 0.0
vec[2] = 0.0
vec[3] = 0.7071067811865474
